In [1]:
using Distributed
addprocs(44);

In [2]:
using pulse_input_DDM

In [3]:
path = "/mnt/bucket/labs/brody/vdtang/P055_processed"
ratnames=["P055"]
sessids=[[17061800,18041700,16052600,16070500,16111700,18010300,18022600,17122200,
        17081200,17092600,17051900,18030500,17011800,18012200,17011500,17062500,
        17112700,18032700,16082200,18060800,18032500,17082900,17080500,17081000,
        17052300,17090800,17090900,18041500,18040500,17122000]];

In [4]:
#parameters of the latent model
pz = Dict("name" => vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    "fit" => vcat(trues(7)),
    "initial" => vcat(2.,15.,-5.,100.,2.,0.2,0.005),
    "lb" => [eps(), 4., -5., eps(), eps(), eps(), eps()],
    "ub" => [10., 100, 5., 800., 40., 2., 10.])

#parameters for the choice observation
pd = Dict("name" => vcat("bias","lapse"), "fit" => trues(2), 
    "initial" => vcat(0.,0.5));

In [5]:
data = aggregate_choice_data(path, sessids, ratnames);

In [6]:
#bin the click inputs at 10 ms
data = bin_clicks!(data;dt=1e-2);

In [ ]:
@time pz, pd = load_and_optimize(data;f_tol=1e-9)

Iter     Function value   Gradient norm 
     0     5.198612e+03     1.179490e+03
     1     5.062700e+03     7.092697e+02
     2     4.866877e+03     1.073818e+02
     3     4.862412e+03     8.480257e+01
     4     4.854455e+03     7.851770e+01
     5     4.843588e+03     2.548006e+01
     6     4.842185e+03     2.133754e+01
     7     4.839475e+03     4.747778e+01
     8     4.834006e+03     5.577361e+01
     9     4.828484e+03     5.779659e+01
    10     4.824905e+03     5.678582e+01
    11     4.816160e+03     6.422642e+01
    12     4.794884e+03     4.350633e+01
    13     4.794010e+03     5.807570e+01
    14     4.789842e+03     5.181033e+01
    15     4.788941e+03     3.346028e+01
    16     4.785648e+03     6.769594e+01
    17     4.782202e+03     4.453928e+01
    18     4.778949e+03     1.973984e+01
    19     4.776610e+03     1.268894e+01
    20     4.774120e+03     2.688848e+01
    21     4.772299e+03     4.290694e+01
    22     4.771867e+03     2.537150e+01


In [ ]:
#compute the Hessian of the LL landscape, to compute confidence intervals on the parameters
pz, pd = compute_H_CI!(pz, pd, data);

In [ ]:
using DataFrames
show(DataFrame(pz),allcols=true)
show(DataFrame(pd),allcols=true)

In [ ]:
#simulate choices from the model, given the ML parameters
ML_data = deepcopy(data)
sample_choices_all_trials!(ML_data, pz["final"], pd["final"])

#compute the final click difference, which will dictate the correct choice
ΔLR = map((nT,L,R)->diffLR(nT,L,R,data["dt"])[end], data["nT"], data["leftbups"], data["rightbups"]);

In [ ]:
#bin the choices from the generative parameters and from the ML parameters
import Pandas: qcut
import Statistics: mean

nbins = 15;
conds,qcut_bins = qcut(ΔLR, nbins, labels=false, retbins=true);
conds = conds .+ 1;

frac_choice_ML = [mean(ML_data["pokedR"][conds .== i]) for i in 1:nbins]
frac_choice_gen = [mean(data["pokedR"][conds .== i]) for i in 1:nbins];

In [ ]:
#fit a GLM to the generative choices and the ML choices
using GLM
GLM_gen = glm(@formula(Y ~ X), DataFrame(X=ΔLR, Y = data["pokedR"]), Binomial(), LogitLink())
GLM_ML = glm(@formula(Y ~ X), DataFrame(X=ΔLR, Y = ML_data["pokedR"]), Binomial(), LogitLink());

In [ ]:
using PyPlot

fig = figure(figsize=(6,6))
ax = subplot(111)

scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice_gen, color="red", label="generative")
scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice_ML, color="grey", label="ML")

plot(sort(ΔLR), sort(predict(GLM_gen)), color="red")
plot(sort(ΔLR), sort(predict(GLM_ML)), color="grey")

ylabel("% poked R")
xlabel(L"\Delta{LR}")
ax[:spines]["top"][:set_color]("none") 
ax[:spines]["right"][:set_color]("none")
legend()

In [ ]:
sessidsF = [[17061801,18041701,16052601,16070501,16111701,18010301,
        18022601,17122201,17081201,17092601,17051901,18030501,
        17011801,18012201,17011501,17062501,17112701,18032701,16082201,
        18060801,18032501,17082901,17080501,17081001,
        17052301,17090801,17090901,18041501,18040501,17122001]];

In [ ]:
#parameters of the latent model
pzF = Dict("name" => vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    "fit" => vcat(trues(7)),
    "initial" => vcat(2.,15.,-5.,100.,2.,0.2,0.005),
    "lb" => [eps(), 4., -5., eps(), eps(), eps(), eps()],
    "ub" => [10., 100, 5., 800., 40., 2., 10.])

#parameters for the choice observation
pdF = Dict("name" => vcat("bias","lapse"), "fit" => trues(2), 
    "initial" => vcat(0.,0.5));

In [ ]:
dataF = aggregate_choice_data(path, sessidsF, ratnames)

In [ ]:
#bin the click inputs at 10 ms
dataF = bin_clicks!(dataF;dt=1e-2);

In [ ]:
@time pzF, pdF = load_and_optimize(dataF)

In [31]:
JLD2.@save "/usr/people/briandd/Projects/pulse_input_DDM.jl/data/results/marino.jld" pz pd pzF pdF